In [ ]:
# Imports 

# Locally created packages
import processing as prc
import configs as cfg

# Global python packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import confusion_matrix, roc_auc_score, plot_roc_curve, auc
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from scipy import stats

In [ ]:
# Create dataframe from csv file
df = pd.read_csv(cfg.path, index_col = 0)

In [ ]:
# Add name of column indicating if a landing is friction limited 
fricLim = cfg.fricLim

# Add name of colmn with the friction coefficient
mu = cfg.mu
        
# Set all values over 0 to 1 (some data have 1-3 indicating how sure one are of the friction limitation)    
df.loc[df[fricLim] > 0, fricLim] = 1

# Set landings with friction coefficient below 0.15 as Slippery
df['slippery'] = df[fricLim]
df.loc[df[mu] > 0.15, 'slippery'] = 0

# Create a new index as combination of date and runway, to be a Unique Key Identifier
df['date'] = df.index
df['key'] = df.index.astype(str) + '-' + df['aerodrome'].astype(str)
df = df.set_index('key')

# Remove possibly duplicated rows
df_notoverlap = df[~df.index.duplicated(keep='first')]
print('Dropped {}'.format(len(df)-len(df_notoverlap)) + ' overlapping flight landings')

# Removing friction coefficients below 0.05, as this is inprobable and has a high probabillity of being wrong measurements
df_notoverlap2 = df_notoverlap.loc[~(df_notoverlap.mu_allRaw <= 0.05),:]

# Drop unusable/unecessary columns
df_new = df_notoverlap2.drop(cfg.unusable_columns, axis = 1)
df_new.aerodrome = df_new.aerodrome.astype(int)
df_new.direction = df_new.direction.astype(int)

# Process wind variables (using direction to get wind in/out from right angle)
df_new = prc.wind_pros(df_new)
df_new = df_new.drop('direction', axis = 1)

# Create the explanatory matix, removing the response variable and unecessary wind variables
X = df_new.drop(['slippery', 'across02_in', 'along02_in','dir10_in','ac_calc02_in'], axis = 1)
X['tmp_abs'] = X.tmp.abs()
X['rwy_abs'] = X.rwy.abs()

# Set the response to be fricLim
y = df_new['slippery']

# Set random state
state = cfg.state

In [ ]:
# Setting up a 10-fold crossvalidation
skf = StratifiedKFold(n_splits=2, random_state = state, shuffle = True)

results_auc = {}
results_params = {}
feature_important = {}

num = 0
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

In [ ]:
# Perform 10-fold crossvalidation of XGBoost classification model
for train_index, test_index in skf.split(X, y):
    
    # Split the data into training and test 
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]

    print('Cv with state: ' + str(state))
    
    # Set up XGBoost model
    xgb_model = xgb.XGBClassifier(objective = 'binary:logistic',seed = state, n_jobs = -1, importance_type = 'total_gain')

    parameters = {'n_estimators': stats.randint(50, 250)
                  'learning_rate': stats.uniform(0.01, 0.2),
                  'subsample': stats.uniform(0.3, 0.7),
                  'min_split_loss': stats.uniform(0, 0.4),
                  'reg_lambda': stats.uniform(0, 10)
                 }

    print('Training model')
    
    # Set up randomized search cross validation
    clf_r = RandomizedSearchCV(xgb_model, parameters, n_jobs=-1, 
                       cv=3, scoring='roc_auc',
                       verbose=10, refit=True, random_state = state, n_iter=20)
    # Fit the model
    clf_r.fit(X_train, y_train)

    # Visualize results with confusion matrix and ROC Curve
    y_proba = clf_r.predict_proba(X_test)[:,1]
    score = roc_auc_score(y_test,y_proba)
    print('ROC AUC score: ' + str(score))

    viz = plot_roc_curve(clf_r, X_test, y_test,
                         name='ROC fold {}'.format(num),
                         alpha=0.3, lw=1)
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

    best_parameters = clf_r.best_params_
    print('Best parameters: ' + str(best_parameters))

    y_pred = clf_r.predict(X_test)
    threshold = sum(y_test)/len(y_test)

    y_class = y_proba.copy()
    y_class[y_class<threshold]=0
    y_class[y_class!=0]=1
    y_test = y_test.reset_index(drop = True)

    print(' ')
    conf_matrix = confusion_matrix(y_test, y_class)
    print('     |              Predicted')
    print('-----|------------------------------------')
    print('     |             | Not FricLim | Friclim')
    print('     |------------------------------------')
    print('True | Not Friclim |' + str(conf_matrix[0][0]) + '         | ' + str(conf_matrix[0][1]))
    print('     |------------------------------------')
    print('     |     Friclim |' + str(conf_matrix[1][0]) + '           | ' + str(conf_matrix[1][1]))
    print('     |------------------------------------')
    print(' ')
    error =  y_test-y_class
    print('Accuracy:', (error.value_counts()[0] / len(error)).round(4))
    print('ROC AUC:', (score).round(4))

    results_auc[num] = score
    results_params[num] = best_parameters
    feature_important[num] = clf_r.best_estimator_.feature_importances_
    num += 1

In [ ]:
# Plot a ROC AUC curve with the mean scores from the 10-fold-CV

fig, ax = plt.subplots()

ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(
    mean_fpr,
    mean_tpr,
    color="b",
    label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(
    mean_fpr,
    tprs_lower,
    tprs_upper,
    color="grey",
    alpha=0.2,
    label=r"$\pm$ 1 std. dev.",
)

ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05],
    title="Receiver operating characteristic",
)
ax.legend(loc="lower right")
plt.show()